In [2]:
import mysql.connector
import pandas as pd
import numpy as np

In [ ]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="qaz!@wsx",
    database="work"
)
print(mydb)

In [ ]:
sql="SELECT * FROM \
stock_age_cal_8 \
WHERE \
stock_age > 60 \
AND SKU NOT LIKE 'P%' \
AND SKU NOT LIKE 'B%' \
AND GoodsName NOT LIKE '%快递盒%' \
AND GoodsName NOT LIKE '%防伪标签%' \
AND GoodsName NOT LIKE '%快递袋%' \
AND GoodsName NOT LIKE '%快递箱%' \
AND GoodsName NOT LIKE '%虚拟商品%' \
AND GoodsName NOT LIKE '%包装盒%' \
AND GoodsName NOT LIKE '%牛皮纸袋%' \
AND cal_time>'2017-12-31' \
AND GoodsName NOT LIKE '%配件%' \
"
sql

In [ ]:
mycursor=mydb.cursor()
mycursor.execute(sql)
stockdata=mycursor.fetchall()

In [3]:
goodslist=pd.read_csv('g_goods_goodslist.csv',header = 0,dtype=object,engine='python',encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: 'g_goods_goodslist.csv'

In [ ]:
goodslist.head()

In [ ]:
goodslist_short=goodslist[['GoodsNO','FixCostPrice']]

In [ ]:
goodslist_short.head()

In [ ]:
stockdata=pd.DataFrame(stockdata)

In [ ]:
stockdata.columns=['GoodsNO','GoodsName','Total_in','date','update_time','cal_time','stock_age']

In [ ]:
stockdata.head()

In [ ]:
def merge(data_left,data_right,key):
    left=pd.read_csv(data_left,header = 0,dtype = object,engine='python',encoding='utf-8')
    right=pd.read_csv(data_right,header = 0,dtype = object,engine='python')
    right=right.drop_duplicates(key)
    data_merge=pd.merge(left,right,on=key,how='left')
    return data_merge

In [ ]:
data_merge=pd.merge(stockdata,goodslist_short,on=['GoodsNO'],how='left')

In [ ]:
data_merge.head()

In [ ]:
data_merge['FixCostPrice']=data_merge['FixCostPrice'].apply(pd.to_numeric)

In [ ]:
data_merge['stockDelayCost']=data_merge['Total_in']*data_merge['FixCostPrice']*0.05/30

In [ ]:
data_merge.head()

In [ ]:
type(data_merge['FixCostPrice'][0])

In [ ]:
data_merge['cal_time_month']=data_merge.cal_time.map(lambda x: x.strftime('%Y-%m'))

In [ ]:
data_merge.head()

In [ ]:
data_merge_groupby=data_merge.groupby(['GoodsNO','GoodsName','cal_time_month'],as_index=False)

In [ ]:
data_merge_groupby=data_merge_groupby.aggregate(np.sum)

In [ ]:
data_merge_groupby['GoodsNO'].isnull().value_counts()

In [ ]:
data_merge_groupby.head()

In [ ]:
GoodsInfo =pd.read_csv('goodsinfo0117.csv',header = 0,dtype=object,engine='python')

In [ ]:
GoodsInfoShort=GoodsInfo[['GoodsNO','Company','Dept']]

In [ ]:
GoodsInfoShort.head()

In [ ]:
GoodsInfoShort.shape

In [ ]:
GoodsInfoShort=GoodsInfoShort.drop_duplicates(["GoodsNO"])

In [ ]:
GoodsInfoShort.shape

In [ ]:
GoodsInfoShort.head()

In [ ]:
data_merge_dept=pd.merge(data_merge_groupby,GoodsInfoShort,on=['GoodsNO'],how='left')

In [ ]:
data_merge_groupby.shape

In [ ]:
data_merge_dept.shape

In [ ]:
data_merge_dept.tail()

In [ ]:
data_merge_dept_gb = data_merge_dept.groupby(['Company','Dept','cal_time_month'],as_index=False)

In [ ]:
data_merge_dept_gb=data_merge_dept_gb.aggregate(np.sum)
data_merge_dept_gb=data_merge_dept_gb[['Company','Dept','cal_time_month','stockDelayCost']]

In [ ]:
data_merge_dept_gb.head()

In [ ]:
data_merge_dept_gb.to_excel("2018库存占用带小组.xlsx",header=True,index=False)